# Scrapbook

This notebook is used for development, its content can change at any time.

In [3]:
import timeatlas as ta
import pandas as pd

In [2]:
s = pd.read_csv("../data/bbdata/2902.csv")
s = DataFrame(data=s["value"].values, index=pd.to_datetime(s["timestamp"]).values)
ts = TimeSeries(s)

## Define a Metadata object

In [ ]:
from timeatlas import Metadata, types

# You can use some typed metadata object
my_unit = types.Unit("power", "W", "float")
my_sensor = types.Sensor(2902, "HB/floor2/22-23C/Prises_Power_Tot")

# Or use Python dictionaries
my_location = {
    "building" : "Blue Factory",
    "floor" : "12",
    "room"  : "22C"
}
my_coordinates = {
    "lat" : 46.796611,
    "lon" : 7.147563
}
my_dict = {
    "unit": my_unit,
    "sensor": my_sensor,
    "location": my_location,
    "coordinates": my_coordinates
}

# Create the Metadata object
my_meta = Metadata(my_dict)


## Define a TimeSeries with Metadata

In [ ]:
ts_meta = TimeSeries(s, my_meta)
ts_meta

## Make Predictions

Some definitions:

- `time series` : self-explanatory, i.e. the TimeSeries object
- `horizon` : the duration to predict after the last value of the time series
- `frequency`: the number of values per unit of time. Usually, the frequency is given in Pandas offset aliases (https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases)

```
                                horizon
   |-------------------------|- - - - - -|
                             |||||||||||||
           time series         frequency

```

In [ ]:
from timeatlas import models

In [ ]:
m1 = models.Prophet()
m1.fit(ts_meta)
pts1 = m1.predict(ts_meta)
pts2 = m1.predict("24 hours")

In [ ]:
m2 = models.LinearRegression()
m2.fit(ts_meta)
pts3 = m2.predict(ts_meta)
pts4 = m2.predict("14 days")

In [ ]:
from timeatlas import plots

plots.prediction(ts_meta, pts1)

## Detect Anomalies


In [ ]:
from timeatlas import detectors, metrics

In [ ]:
ts.apply(lambda x: x+1).series.head()

In [ ]:
# Create the base of the detector. 
# i.e. this detector is based on this model and compute this error
d = detectors.Surprise(m1, metrics.relative_error)

# Set the alerts
d.alerts("quantile", [0.65, 0.8, 0.85, 0.95])\

# Fit the detector with a specific Time Series
d.fit(ts_meta)

In [ ]:
# Detect anomalies with the fitted detector
a = d.detect(ts2)

In [ ]:
a.plot()

In [ ]:
d.truth.plot()
d.prediction.plot()

## Scalers

In [ ]:
from timeatlas.processing import scalers

In [ ]:
ts_meta.plot()

In [ ]:
y = scalers.minmax(ts_meta)

In [ ]:
y.plot()